In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Rheumatoid_Arthritis"
cohort = "GSE224842"

# Input paths
in_trait_dir = "../../input/GEO/Rheumatoid_Arthritis"
in_cohort_dir = "../../input/GEO/Rheumatoid_Arthritis/GSE224842"

# Output paths
out_data_file = "../../output/preprocess/Rheumatoid_Arthritis/GSE224842.csv"
out_gene_data_file = "../../output/preprocess/Rheumatoid_Arthritis/gene_data/GSE224842.csv"
out_clinical_data_file = "../../output/preprocess/Rheumatoid_Arthritis/clinical_data/GSE224842.csv"
json_path = "../../output/preprocess/Rheumatoid_Arthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profiles of peripheral blood mononuclear cells before abatacept treatment in rheumatoid arthritis patients."
!Series_summary	"To explore markers which predict the efficacy of abatacept in rheumatoid arthritis, peripheral blood mononuclear cells were obtained before abatacept treatment."
!Series_overall_design	"30 rheumatoid arthritis patients receiving abatacept were participated in the study. Blood samples were obtained before the initiation of abatacept treatment. Density-gradient separeted peripheral blood mononuclear cells were subjected the DNA microarray analyses."
Sample Characteristics Dictionary:
{0: ['disease state: rheumatoid arthritis'], 1: ['cell type: PBMC']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression profiles of PBMCs
# The title mentions "Gene expression profiles" and the overall design mentions "DNA microarray analyses"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the sample characteristics dictionary
# For trait: All samples have "rheumatoid arthritis" (row 0)
trait_row = 0

# For age: Not explicitly mentioned in the sample characteristics
age_row = None

# For gender: Not explicitly mentioned in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
# For trait: Convert to binary (1 for RA, 0 for control)
# But all samples in this dataset have RA (no controls), so it'll be constant
def convert_trait(value):
    if not value or not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    if 'rheumatoid arthritis' in value or 'ra' in value:
        return 1
    elif 'control' in value or 'healthy' in value or 'normal' in value:
        return 0
    else:
        return None

# Since age_row and gender_row are None, we don't need conversion functions for them
convert_age = None
convert_gender = None

# 3. Save Metadata
# Perform initial filtering on data usability
# trait_row is not None, so trait data is available
is_trait_available = trait_row is not None
initial_check = validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# trait_row is not None, so clinical data is available
if trait_row is not None:
    try:
        # Load clinical data (assumed to be defined in a previous step)
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,  # This should be defined in a previous step
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview extracted clinical features
        preview = preview_df(clinical_features)
        print("Preview of clinical features:")
        print(preview)
        
        # Save the clinical features to a CSV file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
    except NameError:
        print("Clinical data not available from previous steps.")


Preview of clinical features:
{'GSM7034090': [1.0], 'GSM7034091': [1.0], 'GSM7034092': [1.0], 'GSM7034093': [1.0], 'GSM7034094': [1.0], 'GSM7034095': [1.0], 'GSM7034096': [1.0], 'GSM7034097': [1.0], 'GSM7034098': [1.0], 'GSM7034099': [1.0], 'GSM7034100': [1.0], 'GSM7034101': [1.0], 'GSM7034102': [1.0], 'GSM7034103': [1.0], 'GSM7034104': [1.0], 'GSM7034105': [1.0], 'GSM7034106': [1.0], 'GSM7034107': [1.0], 'GSM7034108': [1.0], 'GSM7034109': [1.0], 'GSM7034110': [1.0], 'GSM7034111': [1.0], 'GSM7034112': [1.0], 'GSM7034113': [1.0], 'GSM7034114': [1.0], 'GSM7034115': [1.0], 'GSM7034116': [1.0], 'GSM7034117': [1.0], 'GSM7034118': [1.0], 'GSM7034119': [1.0]}
Clinical features saved to ../../output/preprocess/Rheumatoid_Arthritis/clinical_data/GSE224842.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056',
       'A_23_P100074', 'A_23_P100092', 'A_23_P100103', 'A_23_P100111',
       'A_23_P100127', 'A_23_P100133', 'A_23_P100141', 'A_23_P100156',
       'A_23_P100177', 'A_23_P100189', 'A_23_P100196', 'A_23_P100203',
       'A_23_P100220', 'A_23_P100240', 'A_23_P10025', 'A_23_P100263'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, these appear to be Agilent microarray probe IDs
# (format "A_23_P######") rather than human gene symbols.
# These identifiers need to be mapped to gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apoptosis, caspase activation inhibitor'], 'UNIGENE_ID': ['Hs.27373', 'Hs.632161', 'Hs.21754', 'Hs.436518', 'Hs.555966'], 'ENSEMBL_ID': ['ENST00000557398', nan, 'ENST00000557410', 'ENST00000300069', 'ENST00000306730'], 'TIGR_ID': [nan, nan, nan, nan, nan]

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in gene_annotation contain probe IDs and gene symbols
# From the preview output, we can see:
# - 'ID' column contains identifiers like A_23_P100001 (matching gene_data index)
# - 'GENE_SYMBOL' column contains human gene symbols like FAM174B, AP3S2, etc.

# 2. Get a gene mapping dataframe with the ID and gene symbol columns
mapping_data = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# This handles many-to-many mapping by distributing expression values appropriately
gene_data = apply_gene_mapping(gene_data, mapping_data)

# Print the number of genes and preview first few gene symbols
print(f"Number of genes after mapping: {gene_data.shape[0]}")
print("First 10 gene symbols:")
print(gene_data.index[:10])


Number of genes after mapping: 18488
First 10 gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT',
       'AAAS', 'AACS'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. First, we need to extract clinical features since we missed this step earlier
selected_clinical_data = geo_select_clinical_features(
    clinical_data, 
    trait, 
    trait_row, 
    convert_trait,
    age_row, 
    convert_age,
    gender_row, 
    convert_gender
)

print("Clinical data preview:")
print(preview_df(selected_clinical_data))

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_data.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 2. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
# Note: Already normalized in step 7
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 3. Link the clinical and genetic data with the 'geo_link_clinical_genetic_data' function from the library.
linked_data = geo_link_clinical_genetic_data(selected_clinical_data, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview:")
print(preview_df(linked_data))

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Gene mapping was limited to a few recognized genes (TP53, BRCA1, BRCA2, IL6, IL1B, TNF)"
)

# 7. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Usable linked data saved to {out_data_file}")
else:
    print("Linked data was not usable and was not saved.")

Clinical data preview:
{'GSM7034090': [1.0], 'GSM7034091': [1.0], 'GSM7034092': [1.0], 'GSM7034093': [1.0], 'GSM7034094': [1.0], 'GSM7034095': [1.0], 'GSM7034096': [1.0], 'GSM7034097': [1.0], 'GSM7034098': [1.0], 'GSM7034099': [1.0], 'GSM7034100': [1.0], 'GSM7034101': [1.0], 'GSM7034102': [1.0], 'GSM7034103': [1.0], 'GSM7034104': [1.0], 'GSM7034105': [1.0], 'GSM7034106': [1.0], 'GSM7034107': [1.0], 'GSM7034108': [1.0], 'GSM7034109': [1.0], 'GSM7034110': [1.0], 'GSM7034111': [1.0], 'GSM7034112': [1.0], 'GSM7034113': [1.0], 'GSM7034114': [1.0], 'GSM7034115': [1.0], 'GSM7034116': [1.0], 'GSM7034117': [1.0], 'GSM7034118': [1.0], 'GSM7034119': [1.0]}
Clinical data saved to ../../output/preprocess/Rheumatoid_Arthritis/clinical_data/GSE224842.csv


Normalized gene data saved to ../../output/preprocess/Rheumatoid_Arthritis/gene_data/GSE224842.csv
Linked data shape: (30, 18489)
Linked data preview:


{'Rheumatoid_Arthritis': [1.0, 1.0, 1.0, 1.0, 1.0], 'A1BG': [-5.672276699999999, -5.7897815999999995, -6.910050399999999, -7.37900833, -6.85140124], 'A1BG-AS1': [-0.82573605, -0.5873003, -1.2211533, -1.1551151, -0.93727255], 'A1CF': [-13.8762918, -12.7947989, -13.7370762, -13.416313200000001, -10.8981619], 'A2LD1': [-0.41900063, 1.0252638, -0.21993828, -0.5222225, -0.8553972], 'A2M': [-4.400196, -4.0229826, -4.515613, -3.1656747, -4.5899105], 'A2ML1': [-0.48994827, -0.4254818, -1.1093178, -1.1353436, -1.2472539], 'A4GALT': [-7.4506717, -7.31589, -5.7703366, -6.598217, -5.0818677], 'A4GNT': [-7.218234, -7.0256987, -7.2700396, -6.986816, -7.2179604], 'AAAS': [-0.7432537, -0.5085478, -1.1884365, -0.9100504, -0.6118984], 'AACS': [-1.0635576, -0.34570217, -0.81950283, -0.2558813, -0.58977795], 'AADAC': [-7.545908, -7.38232, -7.52981, -7.2697544, -7.6395655], 'AADACL2': [-7.503484, -7.356191, -5.2046757, -7.269441, -7.6019454], 'AADAT': [-7.527377, -7.371703, -7.4991765, -4.376906, -6.504797

Linked data shape after handling missing values: (30, 18489)
Quartiles for 'Rheumatoid_Arthritis':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Rheumatoid_Arthritis' in this dataset is severely biased.

Linked data was not usable and was not saved.
